<a href="https://colab.research.google.com/github/DavidSenseman/BIO1173/blob/main/Onnx_1A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# You must run this cell first
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    from google.colab import auth
    auth.authenticate_user()
    COLAB = True
    print("Note: Using Google CoLab")
    import requests
    gcloud_token = !gcloud auth print-access-token
    gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
    print(gcloud_tokeninfo['email'])
except:
    print("**WARNING**: Your GMAIL address was **not** printed in the output below.")
    print("**WARNING**: You will NOT receive credit for this lesson.")
    COLAB = False

Mounted at /content/drive
Note: Using Google CoLab
david.senseman@gmail.com


!pip install numpy==1.26.4

In [2]:
!pip install numpy==1.26.4

Step 1: Install Required Packages

In [3]:
# Uninstall the existing version
!pip uninstall -y tf2onnx

# Reinstall from GitHub
!pip install git+https://github.com/onnx/tensorflow-onnx

# Verify version
import tf2onnx
print(tf2onnx.__version__)


Found existing installation: tf2onnx 1.16.1
Uninstalling tf2onnx-1.16.1:
  Successfully uninstalled tf2onnx-1.16.1
  Cloning https://github.com/onnx/tensorflow-onnx to /tmp/pip-req-build-kd34ecwg
  Running command git clone --filter=blob:none --quiet https://github.com/onnx/tensorflow-onnx /tmp/pip-req-build-kd34ecwg
  Resolved https://github.com/onnx/tensorflow-onnx to commit c34ac1d751427cf5d98023a21cce4c82b0cf96a1
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'tf2onnx' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'tf2onnx'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for tf2onnx: filename=tf2onnx-1.16.1-py3-none-any.whl size=45668

Step 2: Copy Model from Google Drive

In [4]:
# Import required libraries
import os
import shutil

# Define the path to your model in Google Drive
gdrive_model_path = '/content/drive/MyDrive/ResNet50_model_244.keras'

# Copy the model to current working directory
local_model_path = '/content/ResNet50_model_244.keras'
shutil.copy(gdrive_model_path, local_model_path)

print(f"Model copied from Google Drive to: {local_model_path}")


Model copied from Google Drive to: /content/ResNet50_model_244.keras


Step 3: Load the Keras Model


In [ ]:
# Import TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras

print("Loading Keras model...")
model = keras.models.load_model(local_model_path)
print("Model loaded successfully!")

# Display model information
print(f"Model summary:")
model.summary()


Step 4: Get Model Input Shape and Create Test Data

In [6]:
# Get model input shape for testing
input_shape = model.input_shape
print(f"\nInput shape: {input_shape}")

# Create sample input data for testing
import numpy as np

# Generate random test data matching your model's expected input
if len(input_shape) == 4:  # Image input
    batch_size = 1
    height, width, channels = input_shape[1], input_shape[2], input_shape[3]
    test_input = np.random.random((batch_size, height, width, channels)).astype(np.float32)
else:
    # For other input types, adjust accordingly
    test_input = np.random.random(input_shape).astype(np.float32)

print(f"Test input shape: {test_input.shape}")



Input shape: (None, 244, 244, 3)
Test input shape: (1, 244, 244, 3)


Step 5: Convert Keras Model to ONNX Format

In [7]:
# Import tf2onnx for conversion
import tf2onnx

print("Converting Keras model to ONNX format...")

try:
    # Create the input specification for conversion
    spec = (tf.TensorSpec(input_shape, tf.float32, name="input"),)

    # Convert the model to ONNX
    output_path = "/content/ResNet50_model_244.onnx"

    onnx_graph = tf2onnx.convert.from_keras(
        model,
        input_signature=spec,
        output_path=output_path,
        opset=13  # Use ONNX opset version 13
        #enable_onnx_checker=False  # Skip ONNX checker to avoid potential issues
    )

    print(f"Model converted successfully!")
    print(f"ONNX model saved to: {output_path}")

except Exception as e:
    print(f"Conversion failed with error: {e}")

Converting Keras model to ONNX format...
Model converted successfully!
ONNX model saved to: /content/ResNet50_model_244.onnx


In [8]:
# Verify ONNX model
try:
    import onnx

    # Load and check the ONNX model
    onnx_model_loaded = onnx.load(output_path)
    onnx.checker.check_model(onnx_model_loaded)
    print("ONNX model validation successful!")

    # Display ONNX model information
    print("\nONNX Model Information:")
    print(f"Model name: {onnx_model_loaded.graph.name}")
    print(f"Number of inputs: {len(onnx_model_loaded.graph.input)}")
    print(f"Number of outputs: {len(onnx_model_loaded.graph.output)}")

except Exception as e:
    print(f"ONNX model validation failed: {e}")

# Run inference using ONNX Runtime
import onnxruntime as ort

print("\nSetting up ONNX Runtime session...")
try:
    session_options = ort.SessionOptions()
    session_options.log_verbosity_level = 0  # Reduce logging verbosity

    # Create ONNX Runtime session
    ort_session = ort.InferenceSession(output_path, session_options)

    # Get input and output names
    input_name = ort_session.get_inputs()[0].name
    output_name = ort_session.get_outputs()[0].name

    print(f"Input name: {input_name}")
    print(f"Output name: {output_name}")

    # Run inference using ONNX Runtime
    print("\nRunning inference with ONNX Runtime...")
    ort_outputs = ort_session.run([output_name], {input_name: test_input})

    print("ONNX Runtime inference completed successfully!")
    print(f"Output shape: {ort_outputs[0].shape}")
    print(f"First few output values: {ort_outputs[0][0][:5]}")

    # Compare with original Keras model
    print("\nComparing results between Keras and ONNX Runtime:")
    keras_output = model.predict(test_input)

    # Calculate difference
    difference = np.abs(keras_output - ort_outputs[0])
    max_diff = np.max(difference)
    mean_diff = np.mean(difference)

    print(f"Max absolute difference: {max_diff}")
    print(f"Mean absolute difference: {mean_diff}")

    # If difference is small, the conversion was successful
    if max_diff < 1e-4:
        print("Conversion and inference successful! Models produce nearly identical results.")
    else:
        print("Small differences detected, but conversion likely successful.")

except Exception as e:
    print(f"ONNX Runtime inference failed: {e}")
    print("This might be due to model complexity or ONNX version incompatibilities.")

# Display inference time comparison
try:
    import time

    # Time Keras inference
    start_time = time.time()
    keras_pred = model.predict(test_input)
    keras_time = time.time() - start_time

    # Time ONNX Runtime inference
    start_time = time.time()
    ort_pred = ort_session.run([output_name], {input_name: test_input})
    ort_time = time.time() - start_time

    print(f"\nInference Time Comparison:")
    print(f"Keras model: {keras_time:.6f} seconds")
    print(f"ONNX Runtime: {ort_time:.6f} seconds")

except Exception as e:
    print(f"Error during timing comparison: {e}")

print("\nDemo completed successfully!")

ONNX model validation successful!

ONNX Model Information:
Model name: tf2onnx
Number of inputs: 1
Number of outputs: 1

Setting up ONNX Runtime session...
Input name: input
Output name: dense_1

Running inference with ONNX Runtime...
ONNX Runtime inference completed successfully!
Output shape: (1, 5)
First few output values: [0.8828796  0.04820953 0.06018713 0.00569142 0.00303225]

Comparing results between Keras and ONNX Runtime:
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
Max absolute difference: 1.1920928955078125e-07
Mean absolute difference: 5.774199962615967e-08
Conversion and inference successful! Models produce nearly identical results.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step

Inference Time Comparison:
Keras model: 0.384204 seconds
ONNX Runtime: 0.174383 seconds

Demo completed successfully!


In [9]:
!ls

drive  ResNet50_model_244.keras  ResNet50_model_244.onnx  sample_data
